# 0.1 WavesExecuter

---

## Basic Usage

This `QurriumInstance` does not any post-processing feature but provides a workdlow that can execute multiple at once.


### a. Import the instances


In [1]:
from qurry import WavesExecuter

experiment_workflow = WavesExecuter()

### b. Preparing quantum circuit

Remember for any quantum circuit, you need to make sure that they all contain classical registers.
Otherwise, `WavesExecuter` will raise an `ValueError` when you try to execute the circuit.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


circuits_dict = {}

for i in range(2, 13, 2):
    circuits_dict[f"trivial_paramagnet_{i}_ms"] = TrivialParamagnet(i)
    circuits_dict[f"ghz_{i}_ms"] = GHZ(i)
    circuits_dict[f"neel_{i}_ms"] = make_neel_circuit(i)

for name, circuit in circuits_dict.items():
    circuit.measure_all()

In [3]:
no_creq = TrivialParamagnet(2)
print(no_creq.draw())

try:
    experiment_workflow.measure([no_creq], shots=1024)
except ValueError as e:
    print(f"Error: {e}")

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘
Error: | No classical register in ALL circuits, counts will be empty. Please add classical register to the circuit. (Don't be frustrated, I did the same thing on unit test. It made me confused and thought what's wrong for a while before ('_').)


### c. Execute the circuit


#### i. Directly input the circuit

After executing, it will return a uuid of experiment. You can use this uuid to get the result of the experiment.


In [4]:
exp1 = experiment_workflow.measure(list(circuits_dict.values()), shots=1024)
exp1

'5118eb7c-03cc-4bf6-9ffb-e03b6423cf33'

Each experiment result will be stored in a container `.exps`.


In [5]:
experiment_workflow.exps[exp1]

<WavesExecuterExperiment(exp_id=5118eb7c-03cc-4bf6-9ffb-e03b6423cf33, 
  WavesExecuterArguments(exp_name='experiment.waves_executer'),
  Commonparams(exp_id='5118eb7c-03cc-4bf6-9ffb-e03b6423cf33', target_keys=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], shots=1024, backend=<AerSimulator('aer_simulator')>, run_args={}, transpile_args={}, tags=(), save_location=PosixPath('.'), serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2025-06-25 13:45:44', 'run.001': '2025-06-25 13:45:44'})),
  unused_args_num=0,
  analysis_num=0))>

`WavesExecuter` does not provide any post-processing feature.
So `.analyze()` **will not provide any useful information**, but

**The Answer to the Ultimate Question of Life, The Universe, and Everything**


In [6]:
report01 = experiment_workflow.exps[exp1].analyze()
report01

| ultimate_question: 


<WavesQurryAnalysis(
  serial=0,
  WEAnalysisInput(ultimate_question=''),
  WEAnalysisContent(ultimate_answer=42, dummy=-100)),
  unused_args_num=0
  )>

In [7]:
main01, side_product01 = report01.export()
main01

{'ultimate_answer': 42,
 'dummy': -100,
 'input': {'ultimate_question': ''},
 'header': {'serial': 0, 'datetime': '2025-06-25 13:45:44', 'log': {}}}

#### ii. Add the circuits to container `.waves`, then call them later.

Since we have executed an experiment, the circuit we input in `exp1` is stored in the container `.waves` with serial number `0`.


In [8]:
experiment_workflow.waves

WaveContainer({
  0: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa2408f0>,
  1: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffab6fa40>,
  2: <qurry.recipe.simple.cat.GHZ object at 0x739ffa1fdf40>,
  3: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73a0da416e40>,
  4: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa403e00>,
  5: <qurry.recipe.simple.cat.GHZ object at 0x739ffa2401a0>,
  6: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x739ffa241550>,
  7: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa241640>,
  8: <qurry.recipe.simple.cat.GHZ object at 0x739ffa6e88c0>,
  9: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x739ffb020080>,
  10: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa241a00>,
  11: <qurry.recipe.simple.cat.GHZ object at 0x739ffa217cb0>,
  12: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x739ffa4de360>,
  13: <qurry.re

But we can also add the circuit to the container `.waves` with a custom name.
The name should be unique, otherwise it will be overwritten.
The method `add` will return the actual name of the circuit in the container.


In [9]:
for k, v in circuits_dict.items():
    print("| Circuit:", experiment_workflow.add(v, k), "added.")

| Circuit: trivial_paramagnet_2_ms added.
| Circuit: ghz_2_ms added.
| Circuit: neel_2_ms added.
| Circuit: trivial_paramagnet_4_ms added.
| Circuit: ghz_4_ms added.
| Circuit: neel_4_ms added.
| Circuit: trivial_paramagnet_6_ms added.
| Circuit: ghz_6_ms added.
| Circuit: neel_6_ms added.
| Circuit: trivial_paramagnet_8_ms added.
| Circuit: ghz_8_ms added.
| Circuit: neel_8_ms added.
| Circuit: trivial_paramagnet_10_ms added.
| Circuit: ghz_10_ms added.
| Circuit: neel_10_ms added.
| Circuit: trivial_paramagnet_12_ms added.
| Circuit: ghz_12_ms added.
| Circuit: neel_12_ms added.


If there is a circuit with the same name, it will be replaced by the new one.


In [10]:
print(experiment_workflow.add(circuits_dict["trivial_paramagnet_2_ms"], "ghz_8_ms"))
print(experiment_workflow.waves["ghz_8_ms"])

ghz_8_ms
        ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas: 2/═════════╩══╩═
                 0  1 


Otherwise, you will need to use `replace="duplicate"` to prevent it from being replaced.


In [11]:
duplicated_case01 = experiment_workflow.add(
    circuits_dict["trivial_paramagnet_2_ms"], "ghz_8_ms", replace="duplicate"
)
print(duplicated_case01)
print(experiment_workflow.waves[duplicated_case01])

ghz_8_ms.37
        ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas: 2/═════════╩══╩═
                 0  1 


Now we have prepared the circuit and stored it in the container `.waves`.


In [12]:
experiment_workflow.waves

WaveContainer({
  0: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa2408f0>,
  1: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffab6fa40>,
  2: <qurry.recipe.simple.cat.GHZ object at 0x739ffa1fdf40>,
  3: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x73a0da416e40>,
  4: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa403e00>,
  5: <qurry.recipe.simple.cat.GHZ object at 0x739ffa2401a0>,
  6: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x739ffa241550>,
  7: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa241640>,
  8: <qurry.recipe.simple.cat.GHZ object at 0x739ffa6e88c0>,
  9: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x739ffb020080>,
  10: <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x739ffa241a00>,
  11: <qurry.recipe.simple.cat.GHZ object at 0x739ffa217cb0>,
  12: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x739ffa4de360>,
  13: <qurry.re

Finally, we can execute the circuit and get the result.


In [13]:
exp2 = experiment_workflow.measure(list(circuits_dict.keys()), shots=1024)
exp2

'0aae1729-ee74-4bce-ac2c-3b978758f7c9'

In [14]:
experiment_workflow.exps[exp2]

<WavesExecuterExperiment(exp_id=0aae1729-ee74-4bce-ac2c-3b978758f7c9, 
  WavesExecuterArguments(exp_name='experiment.waves_executer'),
  Commonparams(exp_id='0aae1729-ee74-4bce-ac2c-3b978758f7c9', target_keys=['trivial_paramagnet_2_ms', 'ghz_2_ms', 'neel_2_ms', 'trivial_paramagnet_4_ms', 'ghz_4_ms', 'neel_4_ms', 'trivial_paramagnet_6_ms', 'ghz_6_ms', 'neel_6_ms', 'trivial_paramagnet_8_ms', 'ghz_8_ms', 'neel_8_ms', 'trivial_paramagnet_10_ms', 'ghz_10_ms', 'neel_10_ms', 'trivial_paramagnet_12_ms', 'ghz_12_ms', 'neel_12_ms'], shots=1024, backend=<AerSimulator('aer_simulator')>, run_args={}, transpile_args={}, tags=(), save_location=PosixPath('.'), serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2025-06-25 13:45:56', 'run.001': '2025-06-25 13:45:56'})),
  unused_args_num=0,
  analysis_num=0))>

In [15]:
report02 = experiment_workflow.exps[exp2].analyze(
    "What is the answer to the ultimate question of life, the universe, and everything"
)
report02

| ultimate_question: What is the answer to the ultimate question of life, the universe, and everything


<WavesQurryAnalysis(
  serial=0,
  WEAnalysisInput(ultimate_question='What is the answer to the ultimate question of life, the universe, and everything'),
  WEAnalysisContent(ultimate_answer=42, dummy=-100)),
  unused_args_num=0
  )>

### d. take `counts`


In [19]:
print("| First 2 counts of the second experiment:")

experiment_workflow.exps[exp2].afterwards.counts[:2]

| First 2 counts of the second experiment:


[{'11': 260, '10': 255, '00': 277, '01': 232}, {'11': 508, '00': 516}]

### e. Export them after all


In [17]:
exp1_id, exp1_files_info = experiment_workflow.exps[exp1].write(
    save_location=".",  # where to save files
)
exp1_files_info

{'folder': 'experiment.waves_executer.001',
 'qurryinfo': 'experiment.waves_executer.001/qurryinfo.json',
 'args': 'experiment.waves_executer.001/args/experiment.waves_executer.001.id=5118eb7c-03cc-4bf6-9ffb-e03b6423cf33.args.json',
 'advent': 'experiment.waves_executer.001/advent/experiment.waves_executer.001.id=5118eb7c-03cc-4bf6-9ffb-e03b6423cf33.advent.json',
 'legacy': 'experiment.waves_executer.001/legacy/experiment.waves_executer.001.id=5118eb7c-03cc-4bf6-9ffb-e03b6423cf33.legacy.json',
 'reports': 'experiment.waves_executer.001/reports/experiment.waves_executer.001.id=5118eb7c-03cc-4bf6-9ffb-e03b6423cf33.reports.json'}

---

### Post-Process Availablities and Version Info


In [18]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurrium version: 0.13.0
---------------------------------------------------------------------------
 ### Qurrium Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     Yes    No   